<table style="width:99%">
<tr>    
<td style="text-align: center">
<h1>Lab 04: Assignment</h1>
<h2>Arkadiusz Paterak</h2>
</td>
</tr>
</table>

# Introduction

The goal of this assignment is to get numerical solution for an advection-diffusion problem using upwind scheme, and compare it with an analytical solution.

The solution might model a physical problems like e.g. ... This particular problem...

In [ ]:
import pint
from pint import Quantity
import numpy as np
import scipy.constants
import matplotlib.pyplot as plt
from open_atmos_jupyter_utils import show_plot

from typing import Callable

# Analytic solution

# Numerical integration

# Comparison of methods

# Conclusions